# Black Scholes

Black Scholes se resume a uma formala, muito utilizada no mercado de ações, para´calcular o preço de contrato de **Opções** de call ou put.

![Black Scholes para contrato de call](./data/black_scholes.jpeg)




### Componentes:
- C = Preço da opção de compra
- S₀ = Preço atual da ação
- K = Preço de exercício (strike)
- r = Taxa livre de risco
- T = Tempo até o vencimento (em anos)
- σ = Volatilidade do ativo
- N() = Função de distribuição normal cumulativa
- e = Número de Euler (≈ 2.71828)


## Problema proposto 
Vamos supor que estamos dispostos a paga X fixo por um contrato, aparti dai gostariamos de simular o valor de strike dado que conhecemos as demais variaveis apresentadas.

Para resolver esse problema de inicio temos as seguinte abordagens:
- #### Método da Bisseção
- #### Método de Newton-Raphson

In [3]:

import time

import numpy as np
import scipy.stats as si
import pandas as pd

In [2]:
# Funções auxiliares para os cálculos de Black-Scholes e Newton-Raphson
def black_scholes_call(S, K, T, r, sigma):
    """Calcula o preço da opção de compra usando Black-Scholes."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)

def black_scholes_call_derivative(S, K, T, r, sigma):
    """Calcula a derivada do preço da opção de compra em relação ao strike K."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return -np.exp(-r * T) * si.norm.cdf(d2)

# Método da Bisseção
def bisection_method(S, C, T, r, sigma, tol=1e-6, max_iter=1000):
    K_min, K_max = S / 2, S * 2
    for _ in range(max_iter):
        K_mid = (K_min + K_max) / 2
        C_mid = black_scholes_call(S, K_mid, T, r, sigma)
        
        if abs(C_mid - C) < tol:
            return K_mid
        
        if C_mid < C:
            K_max = K_mid
        else:
            K_min = K_mid
            
    return K_mid  # Retorna o melhor valor encontrado caso não converja

# Método de Newton-Raphson
def newton_raphson_method(S, C, T, r, sigma, tol=1e-6, max_iter=1000):
    K = S  # Estimativa inicial
    for _ in range(max_iter):
        C_calc = black_scholes_call(S, K, T, r, sigma)
        dC_dK = black_scholes_call_derivative(S, K, T, r, sigma)
        
        if abs(C_calc - C) < tol:
            return K
        
        K = K - (C_calc - C) / dC_dK
        
    return K  # Retorna o melhor valor encontrado caso não converja

# Simulação dos testes
np.random.seed(42)  # Para reprodutibilidade
num_tests = 5
test_results = []

for i in range(num_tests):
    # Valores de teste aleatórios para cada parâmetro, mantendo C constante entre os métodos para cada teste
    S = np.random.uniform(50, 150)  # Preço do ativo subjacente
    K_true = np.random.uniform(40, 160)  # Strike real conhecido, que tentamos recalcular
    T = np.random.uniform(0.5, 2)  # Tempo até o vencimento em anos
    r = np.random.uniform(0.01, 0.05)  # Taxa de juros livre de risco
    sigma = np.random.uniform(0.1, 0.4)  # Volatilidade anual
    
    # Preço da opção (C) calculado com o verdadeiro K_true
    C = black_scholes_call(S, K_true, T, r, sigma)

    # Teste do Método da Bisseção
    start_bisec = time.time()
    K_bisection = bisection_method(S, C, T, r, sigma)
    end_bisec = time.time()
    
    # Teste do Método de Newton-Raphson
    start_newton = time.time()
    K_newton = newton_raphson_method(S, C, T, r, sigma)
    end_newton = time.time()
    
    # Armazenar resultados
    test_results.append({
        "Teste": i + 1,
        "K_true": K_true,
        "K_bisection": K_bisection,
        "Bisection Time (s)": end_bisec - start_bisec,
        "K_newton": K_newton,
        "Newton Time (s)": end_newton - start_newton
    })

test_results


[{'Teste': 1,
  'K_true': 154.08571676918996,
  'K_bisection': 154.0857194184829,
  'Bisection Time (s)': 0.004054546356201172,
  'K_newton': np.float64(154.08568486152961),
  'Newton Time (s)': 0.0044345855712890625},
 {'Teste': 2,
  'K_true': 46.970033460183934,
  'K_bisection': 46.9700340178827,
  'Bisection Time (s)': 0.0057523250579833984,
  'K_newton': np.float64(46.97003345482421),
  'Newton Time (s)': 0.0014963150024414062},
 {'Teste': 3,
  'K_true': 156.38918225943934,
  'K_bisection': 104.11689885916047,
  'Bisection Time (s)': 0.16768407821655273,
  'K_newton': np.float64(152.03287559160012),
  'Newton Time (s)': 0.004030466079711914},
 {'Teste': 4,
  'K_true': 76.50906915514453,
  'K_bisection': 76.50907051283744,
  'Bisection Time (s)': 0.003841876983642578,
  'K_newton': np.float64(76.50906915262789),
  'Newton Time (s)': 0.0013704299926757812},
 {'Teste': 5,
  'K_true': 56.73926327824502,
  'K_bisection': 56.73926306439157,
  'Bisection Time (s)': 0.00471186637878418,
  

In [4]:
df = pd.DataFrame(test_results)
df

,Teste,K_true,K_bisection,Bisection Time (s),K_newton,Newton Time (s)
0,1,154.085717,154.085719,0.004055,154.085685,0.004435
1,2,46.970033,46.970034,0.005752,46.970033,0.001496
2,3,156.389182,104.116899,0.167684,152.032876,0.004030
3,4,76.509069,76.509071,0.003842,76.509069,0.001370
4,5,56.739263,56.739263,0.004712,56.739263,0.001014
